# Model & Evaluation

## 1. Load Packages and Datasets

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import os
import re
import spacy
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter
from wordcloud import WordCloud
from bs4 import BeautifulSoup
from IPython.display import display, HTML

# Install extra libraries
#!pip install bs4
#!pip install matplotlib

In [2]:
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [3]:
# Load pre-processed train and test set from EDA notebook
train_df = pd.read_json('./../data/preprocess_train.json')
test_df = pd.read_json('./../data/preprocess_test.json')